In [41]:
!pip install openai --upgrade



In [42]:
from openai import OpenAI
client = OpenAI()

In [44]:
filename = "world_df.csv"
assistant_name = "data-analyst-v0.1"

In [56]:
# Check if file is already uploaded
filelist = client.files.list(purpose="assistants")
filenames =  [x.filename for x in filelist.data]
file_uploaded = True if filename in filenames else False

# Upload a file with an "assistants" purpose or use existing one
if not file_uploaded:
  file = client.files.create(
    file=open(filename, "rb"),
    purpose='assistants'
  )
else:
  for f in filelist:
    if f.filename == filename:
      file = client.files.retrieve(f.id)
      break

In [59]:


# Check if assistant already exists
assistant_list = client.beta.assistants.list()
assistant_names =  [x.name for x in assistant_list.data]
assistant_exists = True if assistant_name in assistant_names else False

if not assistant_exists:
  # Create an assistant using the file ID
  assistant = client.beta.assistants.create(
    name = "data-analyst-v0.1",
    instructions="You are a data analyst",
    model="gpt-4o",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
      "code_interpreter": {
        "file_ids": [file.id]
      }
    }
  )
else:
    for a in assistant_list:
      if a.name == assistant_name:
        assistant = client.beta.assistants.retrieve(a.id)
        break



In [60]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Using the csv file attached, display a graph of 'Year' against 'Anuual CO2 emissions",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

In [48]:
from typing_extensions import override
from openai import AssistantEventHandler

# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.

class EventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

# Then, we use the `stream` SDK helper
# with the `EventHandler` class to create the Run
# and stream the response.

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="create a downloadable file for the graph",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > code_interpreter

import pandas as pd

# Load the file
file_path = '/mnt/data/file-BbviDPC80gnq8Qbw35ARYzmP'
df = pd.read_csv(file_path)

# Display the first few rows of the dataframe
df.head()

output >

  Entity      Code  Year  Annual CO₂ emissions  \
0  World  OWID_WRL  1850           196896030.0   
1  World  OWID_WRL  1851           198804980.0   
2  World  OWID_WRL  1852           207550940.0   
3  World  OWID_WRL  1853           217209250.0   
4  World  OWID_WRL  1854           255138980.0   

   Annual CO₂ emissions including land-use change  \
0                                    2.621609e+09   
1                                    2.711943e+09   
2                                    2.758550e+09   
3                                    2.825733e+09   
4                                    2.886013e+09   

   Annual CO₂ emissions from land-use change  
0                               2.424713e+09  
1                               2.513138e+09  
2                    

In [75]:
import time
# Poll a run until it has completed
# This code is copied directly from the OpenAI Cookbook (MIT Licence).
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run
messages = client.beta.threads.messages.list(thread_id=thread.id)
print(messages)


SyncCursorPage[Message](data=[Message(id='msg_IIZUP7Sw273w3YkCUtJkJLzr', assistant_id=None, attachments=[Attachment(file_id='file-xdEC98eCXjTGOW8rFIufPUrg', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Using the csv file attached, display a graph of 'Year' against 'Anuual CO2 emissions"), type='text')], created_at=1716743317, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_o1rJUgzNyf7mhB9KeC8nrDcC')], object='list', first_id='msg_IIZUP7Sw273w3YkCUtJkJLzr', last_id='msg_IIZUP7Sw273w3YkCUtJkJLzr', has_more=False)


In [73]:
print(client.beta.threads.messages.list(thread.id).data)

[Message(id='msg_IIZUP7Sw273w3YkCUtJkJLzr', assistant_id=None, attachments=[Attachment(file_id='file-xdEC98eCXjTGOW8rFIufPUrg', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Using the csv file attached, display a graph of 'Year' against 'Anuual CO2 emissions"), type='text')], created_at=1716743317, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_o1rJUgzNyf7mhB9KeC8nrDcC')]


In [49]:
filelist = client.files.list(purpose="assistants_output")

In [50]:
image_list = [x for x in filelist.data if "png" in x.filename]

In [51]:
id = image_list[-1].id # the last in the list is the latest

In [52]:
id

'file-T3Ixnm4mpvaWDrdtig9BbswE'

In [53]:
client.files.retrieve(id)

FileObject(id='file-T3Ixnm4mpvaWDrdtig9BbswE', bytes=108251, created_at=1716726960, filename='/mnt/data/annual_co2_emissions.png', object='file', purpose='assistants_output', status='processed', status_details=None)

In [54]:
image_data = client.files.content(id)
image_data_bytes = image_data.read()

with open("./my-image.png", "wb") as file:
    file.write(image_data_bytes)